In [23]:
import numpy as np


def build_time_axis_31day_months():
    month_start = np.array([1 + 31 * k for k in range(12)], dtype=float)
    t_mid = month_start + 15.0
    period = 31 * 12
    return month_start, t_mid, period


def date_to_t(month_start, month_index_0based, day_in_month):
    """
    Convert (month, day) to t using:
      t = month_start[month] + (day - 1)
    """
    return float(month_start[month_index_0based] + (day_in_month - 1))


# Method (1): Piecewise Linear
def piecewise_linear_periodic(t_query, t_nodes, y_nodes, period):

    t_query = float(t_query)

    t_ext = np.concatenate([t_nodes, [t_nodes[0] + period]])
    y_ext = np.concatenate([y_nodes, [y_nodes[0]]])

    if t_query < t_ext[0]:
        t_query += period

    i = np.searchsorted(t_ext, t_query) - 1
    i = max(0, min(i, len(t_ext) - 2))

    t0, t1 = t_ext[i], t_ext[i + 1]
    y0, y1 = y_ext[i], y_ext[i + 1]

    # Linear interpolation
    return y0 + (y1 - y0) * (t_query - t0) / (t1 - t0)

def compute_piecewise_linear_coeffs(t_nodes, y_nodes, period):
    # periodic extension
    t_ext = np.concatenate([t_nodes, [t_nodes[0] + period]])
    y_ext = np.concatenate([y_nodes, [y_nodes[0]]])

    segments = []

    for k in range(len(t_ext) - 1):
        t0, t1 = t_ext[k], t_ext[k + 1]
        y0, y1 = y_ext[k], y_ext[k + 1]

        m = (y1 - y0) / (t1 - t0)
        b = y0 - m * t0

        segments.append({
            "k": k,
            "t_left": t0,
            "t_right": t1,
            "m": m,
            "b": b,
        })

    return segments


# Newton Divided Differences (Method 2)
def newton_divided_differences(x, y):
    x = np.array(x, dtype=float)
    coef = np.array(y, dtype=float).copy()
    n = len(x)

    for j in range(1, n):
        coef[j:n] = (coef[j:n] - coef[j - 1:n - 1]) / (x[j:n] - x[0:n - j])

    return coef


def newton_evaluate(x_nodes, coef, xq):
    x_nodes = np.array(x_nodes, dtype=float)
    xq = float(xq)

    p = coef[-1]
    for k in range(len(coef) - 2, -1, -1):
        p = p * (xq - x_nodes[k]) + coef[k]
    return float(p)


def method2_poly_interp_9pts_using_method1(t_query, t_nodes, y_nodes, period):
    # 9 evenly spaced points in [1, period]
    x = np.linspace(1.0, float(period), 9)

    # y-values generated by Method (1)
    y = np.array([piecewise_linear_periodic(xi, t_nodes, y_nodes, period) for xi in x], dtype=float)

    # Newton coefficients
    coef = newton_divided_differences(x, y)

    # Evaluate at query
    value = newton_evaluate(x, coef, t_query)
    return value, coef, x, y


# Method (3): Degree-4 Least Squares Fit
def fit_degree4_least_squares(t_nodes, y_nodes):
    t = np.array(t_nodes, dtype=float)
    y = np.array(y_nodes, dtype=float)

    # Vandermonde with increasing powers: [1, t, t^2, t^3, t^4]
    A = np.vander(t, N=5, increasing=True)

    # Least squares solve
    a, *_ = np.linalg.lstsq(A, y, rcond=None)
    return a


def eval_poly_increasing_powers(a, t_query):
    t_query = float(t_query)
    powers = np.array([t_query ** k for k in range(len(a))], dtype=float)
    return float(a @ powers)


def main():
    # Monthly averages
    avg = np.array([33, 34, 40, 51, 60, 69, 75, 74, 67, 56, 47, 38], dtype=float)

    month_start, t_mid, period = build_time_axis_31day_months()

    t_feb19 = date_to_t(month_start, month_index_0based=1, day_in_month=19)   # Feb 19
    t_jul4  = date_to_t(month_start, month_index_0based=6, day_in_month=4)    # Jul 4
    t_dec25 = date_to_t(month_start, month_index_0based=11, day_in_month=25)  # Dec 25

    queries = [("Feb 19", t_feb19), ("Jul 4", t_jul4), ("Dec 25", t_dec25)]

    print("=== Time mapping ===")
    for name, tq in queries:
        print(f"{name}: t = {tq:g}")

    # ---------------- Method (1) ----------------
    print("\n=== Method (1): Piecewise linear interpolation of 12 monthly averages ===")
    for name, tq in queries:
        val = piecewise_linear_periodic(tq, t_mid, avg, period)
        print(f"{name}: T ≈ {val:.6f} F")
    # ---- print Method (1) segment coefficients ----
    print("\n=== Method (1) piecewise linear coefficients ===")

    segments = compute_piecewise_linear_coeffs(t_mid, avg, period)

    for seg in segments:
        print(
            f"segment {seg['k']:2d}: "
            f"[{seg['t_left']:.0f}, {seg['t_right']:.0f}]  "
            f"m = {seg['m']:.12e},  "
            f"b = {seg['b']:.12e}"
        )


    # ---------------- Method (2) ----------------
    print("\n=== Method (2): Degree-8 polynomial interpolation through 9 evenly spaced points (y from method 1) ===")

    coef_saved = None
    x_saved = None
    y_saved = None

    for idx, (name, tq) in enumerate(queries):
        val, coef, x_nodes, y_nodes = method2_poly_interp_9pts_using_method1(tq, t_mid, avg, period)

        # Print parameters once
        if idx == 0:
            coef_saved = coef
            x_saved = x_nodes
            y_saved = y_nodes

            print("\nMethod (2) nodes (9 evenly spaced points) and their y-values (from Method 1):")
            for i in range(len(x_saved)):
                print(f"  x{i} = {x_saved[i]:.6f},  y{i} = {y_saved[i]:.6f}")

            print("\n=== Method (2) Newton coefficients ===")
            for i, ci in enumerate(coef_saved):
                print(f"  c{i} = {ci:.12e}")

        print(f"{name}: T ≈ {val:.6f} F")

    # ---------------- Method (3) ----------------
    print("\n=== Method (3): Direct 4th-order polynomial least-squares fit to 12 monthly averages ===")
    a = fit_degree4_least_squares(t_mid, avg)

    print("P4(t) = a0 + a1 t + a2 t^2 + a3 t^3 + a4 t^4")
    for k, ak in enumerate(a):
        print(f"  a{k} = {ak:.12e}")

    for name, tq in queries:
        val = eval_poly_increasing_powers(a, tq)
        print(f"{name}: T ≈ {val:.6f} F")


In [24]:
if __name__ == "__main__":
    main()

=== Time mapping ===
Feb 19: t = 50
Jul 4: t = 190
Dec 25: t = 366

=== Method (1): Piecewise linear interpolation of 12 monthly averages ===
Feb 19: T ≈ 34.580645 F
Jul 4: T ≈ 72.677419 F
Dec 25: T ≈ 36.548387 F

=== Method (1) piecewise linear coefficients ===
segment  0: [16, 47]  m = 3.225806451613e-02,  b = 3.248387096774e+01
segment  1: [47, 78]  m = 1.935483870968e-01,  b = 2.490322580645e+01
segment  2: [78, 109]  m = 3.548387096774e-01,  b = 1.232258064516e+01
segment  3: [109, 140]  m = 2.903225806452e-01,  b = 1.935483870968e+01
segment  4: [140, 171]  m = 2.903225806452e-01,  b = 1.935483870968e+01
segment  5: [171, 202]  m = 1.935483870968e-01,  b = 3.590322580645e+01
segment  6: [202, 233]  m = -3.225806451613e-02,  b = 8.151612903226e+01
segment  7: [233, 264]  m = -2.258064516129e-01,  b = 1.266129032258e+02
segment  8: [264, 295]  m = -3.548387096774e-01,  b = 1.606774193548e+02
segment  9: [295, 326]  m = -2.903225806452e-01,  b = 1.416451612903e+02
segment 10: [326, 